<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Web Scraping

---

## Before Class

#### Install Selenium

Selenium is a headless browser. It allows us to render JavaScript just as a human-navigated browser would.

To install Selenium, use one of the following:
- **Anaconda:** `conda install -c conda-forge selenium`
- **pip:** `pip install selenium`


#### Install GeckoDriver

You will also need GeckoDriver (this assumes you are using Homebrew for Mac): 

**Mac**
- Install [Homebrew](http://brew.sh/)

- ```brew install geckodriver```

#### Install Firefox

Additionally, you will need to have downloaded the [Firefox browser](https://www.mozilla.org/en-US/firefox/new/?utm_source=google&utm_medium=cpc&utm_campaign=Firefox-Brand-US-GGL-Exact&utm_term=firefox&utm_content=A144_A203_A006336&gclid=Cj0KEQjwnPLKBRC-j7nt1b7OlZwBEiQAv8lMLJUyReT6cPzSYdmEA6uD3YDoieuuuusddgAU7XH6smEaAoje8P8HAQ&gclsrc=aw.ds) for the application in this lesson.

## Learning Objectives
- Revisit how to locate elements on a webpage
- Aquire unstructure data from the internet using Beautiful soup.
- Discuss limitations associated with simple requests and urllib libraries
- Introduce Selenium as a solution, and implement a scraper using selenium

## Lesson Guide

- [Introduction](#intro)
- [Building a web scraper](#building-scraper)
- [Retrieving data from the HTML page](#retrieving-data)
    - [Retrieving the restaurant names](#retrieving-names)
    - [Challenge: Retrieving the restaurant locations](#retrieving-locations)
    - [Retrieving the restaurant prices](#retrieving-prices)
    - [Retrieving the restaurant number of bookings](#retrieving-bookings)


- [Introducting Selenium](#selenium)
    - [Running JavaScript before scraping](#selenium-js)
    - [Using regex to only get digits](#selenium-regex)
    - [Challenge: Use Pandas to create a DataFrame of bookings](#challenge-pandas)
    - [Auto-typing using Selenium](#selenium-typing)


- [Summary](#summary)

<a id="intro"></a>
## Introduction

In this codealong lesson, we'll build a web scraper using urllib and BeautifulSoup. We will also explore how to use a headless browser called Selenium.

We'll begin by scraping OpenTable's DC listings. We're interested in knowing the restaurant's **name, location, price, and how many people booked it today.**

OpenTable provides all of this information on this given page: http://www.opentable.com/washington-dc-restaurant-listings

Let's inspect the elements of this page to assure we can find each of the bits of information in which we're interested.

---

<a id="building-scraper"></a>
## Building a web scraper

Now, let's build a web scraper for OpenTable using urllib and Beautiful Soup:

In [1]:
# import our necessary first packages
from bs4 import BeautifulSoup
import requests

In [2]:
# set the url we want to visit
url = "http://www.opentable.com/washington-dc-restaurant-listings"

# visit that url, and grab the html of said page
html = requests.get(url)

At this point, what is in html?

In [3]:
# .text returns the request content in Unicode
html.text[:500]

'           <!DOCTYPE html><html lang="en"><head><meta charset="utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=9; IE=8; IE=7; IE=EDGE"/> <title>Restaurant Reservation Availability</title>    <meta  name="robots" content="noindex" > </meta><link  rel="canonical" href="https://www.opentable.com/washington-dc-restaurant-listings" > </link>      <link rel="shortcut icon" href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon.ico" type="image/x-icon"/><link rel="icon" href="/'

We will need to convert this html objct into a soup object so we can parse it using python and BS4

In [4]:
# convert this into a soup object
soup = BeautifulSoup(html.text, 'html.parser')

<a id="retrieving-data"></a>
### Retrieving data from the HTML page

Let's first find each restaurant name listed on the page we've loaded. How do we find the page location of the restaurant? (Hint: We need to know where in the **HTML** the restaurant element is housed.) In order to find the HTML that renders the restaurant location, we can use Google Chrome's Inspect tool:

> http://www.opentable.com/washington-dc-restaurant-listings

> 1. Visit the URL above. 

> 2. Right-click on an element you are interested in, then choose Inspect (in Chrome). 

> 3. This will open the Developer Tools and show the HTML used to render the selected page element. 

> Throughout this lesson, we will use this method to find tags associated with elements of the page we want to scrape.

See if you can find the restaurant name on the page. Keep in mind there are many restaurants loaded on the page.

In [5]:
# print the restaurant names
soup.find_all(name='span', attrs={'class':'rest-row-name-text'})

[<span class="rest-row-name-text">Dulce Vista</span>,
 <span class="rest-row-name-text">Breitenberg Greens</span>,
 <span class="rest-row-name-text">Skyway</span>,
 <span class="rest-row-name-text">Bauch</span>,
 <span class="rest-row-name-text">Aurelio Schuster</span>,
 <span class="rest-row-name-text">Oren Circles</span>,
 <span class="rest-row-name-text">Drive</span>,
 <span class="rest-row-name-text">Deleniti</span>,
 <span class="rest-row-name-text">128 Weissnat</span>,
 <span class="rest-row-name-text">Nisi Moore</span>,
 <span class="rest-row-name-text">Retta Nicolas</span>,
 <span class="rest-row-name-text">At Gerhold</span>,
 <span class="rest-row-name-text">Flatley</span>,
 <span class="rest-row-name-text">1197 Koch</span>,
 <span class="rest-row-name-text">Qui Turnpike</span>,
 <span class="rest-row-name-text">Mills</span>,
 <span class="rest-row-name-text">Agloe Bar &amp; Grill</span>,
 <span class="rest-row-name-text">Pipers</span>,
 <span class="rest-row-name-text">Laudan

It is important to always keep in mind the data types that were returned. Note this is a `list`, and we know that immediately by observing the outer square brackets and commas separating each tag.

Next, note the elements of the list are `Tag` objects, not strings. (If they were strings, they would be surrounded by quotes.) The Beautiful Soup authors chose to display a `Tag` object visually as a text representation of the tag and its contents. However, being an object, it has many methods that we can call on it. For example, next we will use the `encode_contents()` method to return the tag's contents encoded as a Python string.

<a id="retrieving-names"></a>
#### Retrieving the restaurant names

Now that we found a list of tags containing the restaurant names, let's think how we can loop through them all one-by-one. In the following cell, we'll print out the name (and **only** the clean name, not the rest of the html) of each restaurant.

In [6]:
# for each element you find, print out the restaurant name
for entry in soup.find_all(name='span', attrs={'class':'rest-row-name-text'}):
    print(entry.text)

Dulce Vista
Breitenberg Greens
Skyway
Bauch
Aurelio Schuster
Oren Circles
Drive
Deleniti
128 Weissnat
Nisi Moore
Retta Nicolas
At Gerhold
Flatley
1197 Koch
Qui Turnpike
Mills
Agloe Bar & Grill
Pipers
Laudantium Little
Minus Shores
1018 Ernser
Et Paucek
Rogelios
36 Klocko
Avenue
Corrupti
Garrick Forest
1082 Parker
Mountain
Laborum Stravenue
Orie Buckridge
Rem
199 Stoltenberg
Velit Overpass
Kane Crossing
Cole
Gerlach Viaduct
Lodge
964 Ullrich
Non
Quae
Ea
Officia Cliff
Laborum Spencer
Lockman
Abshire
Flatley Walks
Nathans
Et
Distinctio Street
Aut Anderson
Fugiat Roads
Extensions
Asperiores
Dibbert
Eltas
1458 Zieme
Dickens
Purdy
Molestias Forge
Facilis
1496 Haley
Amelies
Glens
Quia Port
Laudantium Lakes
Soluta
Wyman
141 Jacobs
Christinas
Vero Hills
Dolor Oberbrunner
Lakes
Molestiae Mann
Coralie Willms
Voluptatum Hagenes
Shoals
Wolf Ramp
548 Jenkins
Illum
Derek Crescent
Conn
August Trafficway
194 Ward
Cum
Distinctio Ratke
Chanelle Feeney
Et Hamill
Davin Manors
Arthur Rest
Vivian Lakes
Lolit

Great!



<a id="retrieving-prices"></a>
#### Retrieving the restaurant prices

Ok, we've figured out the restaurant name and location. Now we need to grab the price (number of dollar signs on a scale of one to four) for each restaurant. We'll follow the same process.

In [7]:
# print out all prices
soup.find_all('div', {'class':'rest-row-pricing'})

[<div class="rest-row-pricing"> <i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"> <i>  $    $    $    $  </i> </div>,
 <div class="rest-row-pricing"> <i>  $    $    $    </i>   $        </div>,
 <div class="rest-row-pricing"> <i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"> <i>  $    $    $    $  </i> </div>,
 <div class="rest-row-pricing"> <i>  $    $    $    $  </i> </div>,
 <div class="rest-row-pricing"> <i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"> <i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"> <i>  $    $    $    </i>   $        </div>,
 <div class="rest-row-pricing"> <i>  $    $    $    </i>   $        </div>,
 <div class="rest-row-pricing"> <i>  $    $    $    $  </i> </div>,
 <div class="rest-row-pricing"> <i>  $    $    $    </i>   $        </div>,
 <div class="rest-row-pricing"> <i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"> <i>  $    $ 

In [8]:
# print out EACH number of dollar signs per restaurant
# this one is trickier to eliminate the html. Hint: try a nested find
for entry in soup.find_all('div', {'class':'rest-row-pricing'}):
    print(entry.find('i').text)

  $    $      
  $    $    $    $  
  $    $    $    
  $    $      
  $    $    $    $  
  $    $    $    $  
  $    $      
  $    $      
  $    $    $    
  $    $    $    
  $    $    $    $  
  $    $    $    
  $    $      
  $    $    $    $  
  $    $    $    $  
  $    $    $    $  
  $    $      
  $    $    $    
  $    $    $    $  
  $    $    $    
  $    $    $    
  $    $      
  $    $    $    $  
  $    $    $    $  
  $    $    $    $  
  $    $      
  $    $      
  $    $    $    $  
  $    $    $    $  
  $    $      
  $    $      
  $    $    $    
  $    $    $    $  
  $    $    $    
  $    $    $    $  
  $    $    $    
  $    $      
  $    $    $    $  
  $    $    $    $  
  $    $    $    
  $    $    $    $  
  $    $      
  $    $    $    $  
  $    $      
  $    $    $    $  
  $    $      
  $    $    $    
  $    $    $    $  
  $    $    $    $  
  $    $    $    
  $    $    $    $  
  $    $    $    
  $    $    $    $  
  $    $    $    
 

That looks great, but what if I wanted just the number of dollar signs per restaurant? Can you figure out a way to simply print out the number of dollar signs per restaurant listed?

In [9]:
# print the number of dollars signs per restaurant
for entry in soup.find_all('div', {'class':'rest-row-pricing'}):
    price = entry.find('i').text
    print(price.count('$'))

2
4
3
2
4
4
2
2
3
3
4
3
2
4
4
4
2
3
4
3
3
2
4
4
4
2
2
4
4
2
2
3
4
3
4
3
2
4
4
3
4
2
4
2
4
2
3
4
4
3
4
3
4
3
4
2
3
3
3
4
4
3
2
3
2
4
3
4
2
2
2
4
4
4
3
3
2
3
4
2
3
2
2
2
4
2
4
4
3
2
2
3
4
2
4
4
3
4
3
4


Phew, nice work. 

<a id="retrieving-bookings"></a>
#### Retrieving the restaurant number of bookings

One more, right? We only need to find the number times a restaurant was booked. In the next cell, print out all objects that contain the number of times the restaurant was booked.

In [10]:
# print out all objects that contain the number of times the restaurant was booked
soup.find_all('div', {'class':'booking'})

[]

That's weird -- an empty set. Did we find the wrong element? What's going on here? Discuss.

How can we debug this? Any ideas?

In [13]:
# let's first try printing out all 'div' objects
#  NOTE: This is a too many objects to store in this notebook!
#        So, uncomment the code below to run it.

# for entry in soup.find_all('div'):
#     print entry

I still don't see it. Let's search our entire soup object:

In [14]:
# print out soup, do command+f for "booked ". 
#   Uncomment the below to run.

# soup

What do you notice? Why is this happening?

<a id="selenium"></a>
## Introducing Selenium

Selenium is a headless browser. It allows us to render JavaScript just as a human-navigated browser would.

To install Selenium, use one of the following:
- **Anaconda:** `conda install -c conda-forge selenium`
- **pip:** `pip install selenium`

You will also need GeckoDriver (this assumes you are using Homebrew for Mac): 

- ```brew install geckodriver```

Additionally, you will need to have downloaded the [Firefox browser](https://www.mozilla.org/en-US/firefox/new/?utm_source=google&utm_medium=cpc&utm_campaign=Firefox-Brand-US-GGL-Exact&utm_term=firefox&utm_content=A144_A203_A006336&gclid=Cj0KEQjwnPLKBRC-j7nt1b7OlZwBEiQAv8lMLJUyReT6cPzSYdmEA6uD3YDoieuuuusddgAU7XH6smEaAoje8P8HAQ&gclsrc=aw.ds) for the application in this lesson.

In [11]:
# import
from selenium import webdriver

Selenium requires us to determine a default browser to run. I'm going to opt for Firefox, but Chromium is also a very common choice. http://selenium-python.readthedocs.io/faq.html

In [16]:
# STOP
# what is going to happen when I run the next cell?

In [12]:
# create a driver called Firefox
driver = webdriver.Firefox()

Pretty crazy, right? Let's close that driver.

In [19]:
# close it
#driver.close()

In [13]:
# let's boot it up, and visit a URL of our choice
driver = webdriver.Firefox()
driver.get("http://www.python.org")

Awesome. Now we're getting somewhere: programmatically controlling our browser like a human.

Let's return to our problem at hand. We need to visit the OpenTable listing for DC. Once there, we need to get the html to load. In the next cell, prove you can programmatically visit the page.

In [14]:
# visit our OpenTable page
driver = webdriver.Firefox()
driver.get("http://www.opentable.com/washington-dc-restaurant-listings")

# always good to check we've got the page we think we do
assert "OpenTable" in driver.title

In [15]:
driver.title

'Washington, D.C. Area Restaurants List | OpenTable'

In [16]:
driver.close()

<a id="selenium-js"></a>
### Running JavaScript before scraping

Now, to resolve our JavaScript problem, there's a few things we can do. What I'll do in this case is request that the page load, wait one second, and then I'm going to grab the source html from the page. Because the page should believe I'm visiting from a live connection on a browser client, the JavaScript should render to be a part of the page source. I can then grab the page source.

In [17]:
# import sleep
from time import sleep

In [18]:
# visit our relevant page
driver = webdriver.Firefox()
driver.get("http://www.opentable.com/washington-dc-restaurant-listings")

# wait one second
sleep(1)

#grab the page source
html = driver.page_source

**Pop Quiz:** What do we need to do with this HTML?

In [19]:
# BeautifulSoup it!
html = BeautifulSoup(html, "lxml")

Now, let's return to our earlier problem: How do we locate bookings on the page?

In [20]:
# print out the number bookings for all restaurants
html.find_all('div', {'class':'booking'})

[<div class="booking"><span class="tadpole"></span>Booked 644 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 377 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 238 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 73 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 197 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 141 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 55 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 63 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 92 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 31 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 129 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 40 times today</div>,
 <div class="booking"><span class=

In [21]:
# now print out each booking for the listings using a loop
for entry in html.find_all('div', {'class':'booking'}):
    print(entry)

<div class="booking"><span class="tadpole"></span>Booked 644 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 377 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 238 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 73 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 197 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 141 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 55 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 63 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 92 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 31 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 129 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 40 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 4

Let's grab just the text of each of these entries.

In [22]:
# do the same as above, but grabbing only the text content
for entry in html.find_all('div', {'class':'booking'}):
    print(entry.text)

Booked 644 times today
Booked 377 times today
Booked 238 times today
Booked 73 times today
Booked 197 times today
Booked 141 times today
Booked 55 times today
Booked 63 times today
Booked 92 times today
Booked 31 times today
Booked 129 times today
Booked 40 times today
Booked 44 times today
Booked 27 times today
Booked 38 times today
Booked 28 times today
Booked 63 times today
Booked 84 times today
Booked 83 times today
Booked 69 times today
Booked 129 times today
Booked 47 times today
Booked 118 times today
Booked 19 times today
Booked 65 times today
Booked 45 times today
Booked 36 times today
Booked 25 times today
Booked 74 times today
Booked 40 times today
Booked 119 times today
Booked 52 times today
Booked 75 times today
Booked 23 times today
Booked 73 times today
Booked 32 times today
Booked 18 times today
Booked 36 times today
Booked 11 times today
Booked 89 times today
Booked 16 times today
Booked 23 times today
Booked 48 times today
Booked 15 times today
Booked 27 times today
B

In [31]:
#driver.close()

We've succeeded!

<a id="selenium-regex"></a>
### Using regex to only get digits

But we can clean this up a little bit. We're going to use regular expressions (regex) to grab only the digits that are available in each of the text.

The best way to get good at regex is to, well, just keep trying and testing: http://pythex.org/

In [23]:
# import regex
import re

Given we haven't covered regex, I'll show you how to use the search function to match any given digit.

In [24]:
# for each entry, grab the text
for booking in html.find_all('div', {'class':'booking'}):
    # match all digits
    match = re.search(r'\d+', booking.text)
    
    if match:
        # print if found
        print(match.group())
    else:
        # otherwise pass
        pass

644
377
238
73
197
141
55
63
92
31
129
40
44
27
38
28
63
84
83
69
129
47
118
19
65
45
36
25
74
40
119
52
75
23
73
32
18
36
11
89
16
23
48
15
27
2
46
69
29
22
16
38
44
10
82
45
23
27
38
9
10
12
19
28
16
10
38
35
42
22
11
37
36
15
13
22
35
7
81
33
6
42
36
21
10
5
34
32
35
9
29
21
12
24
14
10
33
29


Before we demonstrate all the other amazing things about headless browsers, let's finish up collecting the data we want from this current example. Do you suppose the html parsing we wrote above will still work on the page source we've grabbed from our headless browser?

To be most efficient, we want to only do a single loop for each entry on the page. That means we want to find what element all of other other elements (name, location, price, bookings) is housed within. Where on the page is each entry located?

In [34]:
# print out all entries
#   NOTE: Has many entries. Uncomment the below code to run it!

# soup.find_all('div', {'class':'result content-section-list-row cf with-times'})

Look over the page. Does every single entry have each element we're seeking?
> I did this previously. I know for a fact that not every element has a number of recent bookings. That's probably exactly why OpenTable houses this in JavaScript: they want to continously update the number of bookings with the most relevant number of values.

In [25]:
# We want to only retrieve the text of the bookings.
# But what would happen if we just naively print the text of each node?

for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    print(entry.find('div', {'class':'booking'}))   # try adding .text

None
None
<div class="booking"><span class="tadpole"></span>Booked 644 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 377 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 238 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 73 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 197 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 141 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 55 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 63 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 92 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 31 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 129 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 40 times today</div>
<div class="booking"><span class="tadpole"></spa

In [36]:
# if we find the element we want, we print it. Otherwise, we print 'ZERO'
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
        divs = entry.find('div', {'class':'booking'})
        
        if divs:
            print(divs.text)
        else:
            print('ZERO')

ZERO
ZERO
Booked 510 times today
Booked 313 times today
Booked 183 times today
Booked 84 times today
Booked 133 times today
Booked 101 times today
Booked 141 times today
Booked 77 times today
Booked 129 times today
Booked 67 times today
Booked 144 times today
Booked 60 times today
Booked 45 times today
Booked 35 times today
Booked 70 times today
Booked 89 times today
Booked 76 times today
Booked 67 times today
Booked 68 times today
Booked 60 times today
Booked 75 times today
Booked 47 times today
Booked 87 times today
Booked 36 times today
Booked 84 times today
Booked 40 times today
Booked 71 times today
Booked 28 times today
Booked 98 times today
Booked 71 times today
Booked 130 times today
Booked 53 times today
Booked 66 times today
Booked 63 times today
Booked 75 times today
Booked 63 times today
Booked 22 times today
Booked 104 times today
Booked 16 times today
Booked 39 times today
Booked 20 times today
Booked 37 times today
Booked 85 times today
Booked 39 times today
Booked 38 ti

What do you notice takes the place when booking is not found?

We could use exception handling (`try`/`except` blocks) to resolve this. However, exceptions should only be used to handle rare or unexpected errors -- never for normal program flow.

In this case, we expect that some entries will be zero. So, we can just use an `if` statement that tests whether there are any `divs` present; if not, display `'ZERO'`. Here's a demo:

In [26]:
# if we find the element we want, we print it. Otherwise, we print 'ZERO'
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    booking_tag = entry.find('div', {'class':'booking'})
    
    if booking_tag:
        print(booking_tag.text)
    else:
        print('ZERO')

ZERO
ZERO
Booked 644 times today
Booked 377 times today
Booked 238 times today
Booked 73 times today
Booked 197 times today
Booked 141 times today
Booked 55 times today
Booked 63 times today
Booked 92 times today
Booked 31 times today
Booked 129 times today
Booked 40 times today
Booked 44 times today
Booked 27 times today
Booked 38 times today
Booked 28 times today
Booked 63 times today
Booked 84 times today
Booked 83 times today
Booked 69 times today
Booked 129 times today
Booked 47 times today
Booked 118 times today
Booked 19 times today
Booked 65 times today
Booked 45 times today
Booked 36 times today
Booked 25 times today
Booked 74 times today
Booked 40 times today
Booked 119 times today
Booked 52 times today
Booked 75 times today
Booked 23 times today
Booked 73 times today
Booked 32 times today
Booked 18 times today
Booked 36 times today
Booked 11 times today
Booked 89 times today
Booked 16 times today
Booked 23 times today
Booked 48 times today
Booked 15 times today
Booked 27 tim

After previously completing this, we observed that all other elements WILL be returned. This means we do not have to always handle these cases.

<a id="challenge-pandas"></a>
### Challenge: Use Pandas to create a DataFrame of bookings

However, the onus is on you to now put all the pieces together.

Loop through each entry. For each entry, grab the relevant information we want (name, location, price, bookings). Produce a dataframe with the columns "name","location","price","bookings" that contains the 100 entries we would like.

In [27]:
import pandas as pd

In [28]:
# I'm going to create my empty df first
dc_eats = pd.DataFrame(columns=["name","location","price","bookings"])

In [29]:
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    # grab the name
    name = entry.find('span', {'class':'rest-row-name-text'}).text
    
    # grab the location 
    location = entry.find('span', {'class':'rest-row-meta--location rest-row-meta-text'}).text
    
    # grab the price
    price = entry.find('div', {'class':'rest-row-pricing'}).find('i').text.count('$')
    
    # try to find the number of bookings
    bookings = 'NA'
    booking_tag = entry.find('div', {'class':'booking'})
    if booking_tag:
        match = re.search('\d+', booking_tag.text)
        
        if match:
            bookings = match.group()
    
    dc_eats.loc[len(dc_eats)]=[name, location, price, bookings]

In [30]:
# check out our work
dc_eats.head()

,name,location,price,bookings
0,Ruffino's - Arlington,Arlington,2,NA
1,Joe's Place Pizza and Pasta,Arlington,2,NA
2,Founding Farmers - DC,Foggy Bottom,2,644
3,Farmers Fishers Bakers,Georgetown,2,377
4,Filomena Ristorante,Georgetown,3,238


Awesome! We succeeded.



The above example (and many others) are available in the Selenium docs: http://selenium-python.readthedocs.io/getting-started.html

What is especially important is exploring functionality like locating elements: http://selenium-python.readthedocs.io/locating-elements.html#locating-elements

FAQ:
http://selenium-python.readthedocs.io/faq.html

### Summary

In this lesson, we used the Beautiful Soup library to locate elements on a website then scrape their text. We also used the Selenium headless browser to run JavaScript first before retrieving the page contents.